### Importing required modules

In [28]:
# import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
import PyPDF2

In [2]:
# loading the env variables
load_dotenv()

True

In [29]:
# loading the google gemini pro LLM model

GOOGLE_API_KEY = os.getenv('API_KEY')
# genai.configure(api_key=GOOGLE_API_KEY)
# model = genai.GenerativeModel('gemini-pro',generation_config={'temperature':0.3})
model = ChatGoogleGenerativeAI(model="gemini-pro",
                               google_api_key=GOOGLE_API_KEY,
                               temperature=0.3)

### Creating Prompt

In [34]:
Template = '''\
You are an expert in the creation of multi choice questions. \
Your job is create a {number} number of multi choice questions in {difficulty} level. \
Make sure the questions are not repeated and follow the below format for creating questions: \n
    1. first multi choice question.\n
    Ans: \n\tA. "first choice here."\n\tB. "second choice here."\n\tC. "third choice here."\n\tD. "fourth choice here."\n
    Correct: "correct answer".\n
    2. second multi choice question.\n
    Ans: \n\tA. "first choice here."\n\tB. "second choice here."\n\tC. "third choice here."\n\tD. "fourth choice here."\n
    Correct: "correct answer".\n
Using following text to generate muti choice questions based above instructions:
"Text" : {text}
'''

In [35]:
print(Template)

You are an expert in the creation of multi choice questions. Your job is create a {number} number of multi choice questions in {difficulty} level. Make sure the questions are not repeated and follow the below format for creating questions: 

    1. first multi choice question.

    Ans: 
	A. "first choice here."
	B. "second choice here."
	C. "third choice here."
	D. "fourth choice here."

    Correct: "correct answer".

    2. second multi choice question.

    Ans: 
	A. "first choice here."
	B. "second choice here."
	C. "third choice here."
	D. "fourth choice here."

    Correct: "correct answer".

Using following text to generate muti choice questions based above instructions:
"Text" : {text}



In [36]:
mcq_prompt = PromptTemplate(
    input_variables=['number','difficulty','text'],
    template=Template
)

In [37]:
base_chain = LLMChain(llm=model,
                      prompt=mcq_prompt,
                      output_key="mcq",
                      verbose=True
                      )